# DocumentLanguageClassifier

In [1]:
%load_ext autoreload
%autoreload 2

## On its own

In [2]:
from haystack import Document
from haystack.components.classifiers import TransformersZeroShotDocumentClassifier

In [3]:
documents = [Document(id="0", content="Cats don't get teeth cavities."),
             Document(id="1", content="Cucumbers can be grown in water.")]

In [4]:
document_classifier = TransformersZeroShotDocumentClassifier(
    model="cross-encoder/nli-deberta-v3-xsmall",
    labels=["animals", "food", "science"],
)

In [5]:
document_classifier.warm_up()
document_classifier.run(documents = documents) 

/mnt/Exdisk/git-cuongpiger/haystack-tutorial/.venv/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cuda:0


{'documents': [Document(id=0, content: 'Cats don't get teeth cavities.', meta: {'classification': {'label': 'animals', 'score': 0.9501339197158813, 'details': {'animals': 0.9501339197158813, 'science': 0.040235087275505066, 'food': 0.009631010703742504}}}),
  Document(id=1, content: 'Cucumbers can be grown in water.', meta: {'classification': {'label': 'food', 'score': 0.5177573561668396, 'details': {'food': 0.5177573561668396, 'science': 0.4288996756076813, 'animals': 0.05334296077489853}}})]}

In [6]:
for doc in documents:
    print(doc)
    print()

Document(id=0, content: 'Cats don't get teeth cavities.', meta: {'classification': {'label': 'animals', 'score': 0.9501339197158813, 'details': {'animals': 0.9501339197158813, 'science': 0.040235087275505066, 'food': 0.009631010703742504}}})

Document(id=1, content: 'Cucumbers can be grown in water.', meta: {'classification': {'label': 'food', 'score': 0.5177573561668396, 'details': {'food': 0.5177573561668396, 'science': 0.4288996756076813, 'animals': 0.05334296077489853}}})



## In pipeline

In [7]:
from haystack import Document
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.core.pipeline import Pipeline
from haystack.components.classifiers import TransformersZeroShotDocumentClassifier

In [8]:
documents = [Document(id="0", content="Hôm nay là một ngày tuyệt vời!"),
             Document(id="1", content="Ngày hôm qua quả là một ngày tệ hại!")]

In [9]:
document_store = InMemoryDocumentStore()
retriever = InMemoryBM25Retriever(document_store=document_store)
document_classifier = TransformersZeroShotDocumentClassifier(
    model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli",
    labels=["tích cực", "tiêu cực"],
)

In [10]:
document_store.write_documents(documents)

2

In [11]:
pipeline = Pipeline()
pipeline.add_component(instance=retriever, name="retriever")
pipeline.add_component(instance=document_classifier, name="document_classifier")
pipeline.connect("retriever", "document_classifier")

🚅 Components
  - retriever: InMemoryBM25Retriever
  - document_classifier: TransformersZeroShotDocumentClassifier
🛤️ Connections
  - retriever.documents -> document_classifier.documents (List[Document])

In [12]:
queries = ["Ngày hôm nay thế nào?", "Ngày hôm qua của bạn như thế nào?"]
expected_predictions = ["tích cực", "tiêu cực"]

In [13]:
for idx, query in enumerate(queries):
    result = pipeline.run({"retriever": {"query": query, "top_k": 1}})
    print(result)
    assert result["document_classifier"]["documents"][0].to_dict()["id"] == str(idx)
    assert (result["document_classifier"]["documents"][0].to_dict()["classification"]["label"]
            == expected_predictions[idx])

Device set to use cuda:0


{'document_classifier': {'documents': [Document(id=0, content: 'Hôm nay là một ngày tuyệt vời!', meta: {'classification': {'label': 'tích cực', 'score': 0.998790979385376, 'details': {'tích cực': 0.998790979385376, 'tiêu cực': 0.0012090412201359868}}}, score: 1.2328974712196277)]}}
{'document_classifier': {'documents': [Document(id=1, content: 'Ngày hôm qua quả là một ngày tệ hại!', meta: {'classification': {'label': 'tiêu cực', 'score': 0.9993325471878052, 'details': {'tiêu cực': 0.9993325471878052, 'tích cực': 0.0006674606702290475}}}, score: 1.2042055389754964)]}}
